# Closest subset-sum with rational numbers

In [1]:
# Data wrangling
import pandas as pd

# ClosestSubsetSum class
import datetime
import logging
from itertools import chain, combinations

In [ ]:
name = "planilha plan 2023"
in_path = "data/" + name + ".xlsx"

df = pd.read_excel(in_path, engine="openpyxl")
df

In [3]:
class ClosestSubsetSum:
    def __init__(self, array: pd.Series, depth=None, target=None):
        if depth:
            self.depth = min(depth, len(array))
        else:
            self.depth = len(array)
        self.array = array.dropna().sort_values(ascending=False).iloc[:depth]
        self._solvers = {"naive": self.naive}
        self.best_combination = None
        self.best_sum = 0
        if target:
            self.target = target
        else:
            self.target = self.array.sum() * 0.7

    @staticmethod
    def timed(solver):
        def time_summary(*args, **kwargs):
            start = datetime.datetime.now()
            print(
                f"Execution started at "
                f"{start - datetime.timedelta(microseconds = start.microsecond)}"
            )
            solver(*args, **kwargs)
            end = datetime.datetime.now()
            print(
                f"Execution ended at "
                f"{end - datetime.timedelta(microseconds = end.microsecond)}"
            )
            print(f"Runtime: {end-start}")

        return time_summary

    def solve(self, solver_name="naive", verbose=True, tolerance=0):
        if solver_name not in self._solvers:
            print(f"Method '{solver_name}' undefined")
            return
        if verbose:
            return self.timed(self._solvers[solver_name])(tolerance)
        else:
            return self._solvers[solver_name](tolerance)

    def naive(self, tolerance: float):
        def powerset(a):
            return chain.from_iterable(combinations(a, r) for r in range(len(a) + 1))

        num_combinations = 2 ** len(self.array)
        threshold = self.target * (1 - tolerance)

        logging.critical(
            f"Number of combinations: {num_combinations} "
            f"Target: {self.target:.2f} "
            f"Tolerance: {tolerance}"
        )
        logging.critical("*" * 40)

        for i, combination in enumerate(powerset(self.array)):
            combo_sum = 0
            progress = f"Progress: {i*100/num_combinations:.2f}% "

            for num in combination:
                combo_sum += num
            if self.target >= combo_sum >= self.best_sum:
                self.best_sum = combo_sum
                self.best_combination = combination

                logging.critical(
                    progress + f"Error: {self.target-self.best_sum:.2f}\n"
                    f"{self.best_combination}"
                )
            # Stop criteria
            if threshold <= self.best_sum:
                break
        logging.critical("*" * 40)

In [ ]:
tar = 30012429.20
tol = 0

logging.basicConfig(
    filename="logs/" + name + f" (naive, {tol}).log",
    filemode="w",
    level=logging.CRITICAL,
    force=True,
    format="[%(asctime)s] %(message)s",
    datefmt="%d-%b-%y %H:%M:%S",
)

css = ClosestSubsetSum(df["Valor"], target=tar)
css.solve(tolerance=tol)

logging.shutdown()

In [ ]:
mask = df.Valor.isin(css.best_combination)

cell_sum = "Sum = " + str(df.Valor[mask].sum())
cell_error = str(round(tar - df.Valor[mask].sum(), 2))

df[cell_sum] = 0
df.loc[mask, cell_sum] = 1
df.iloc[0, -1] = "Error = " + cell_error

df

In [ ]:
# Checksum
df.Valor[mask].sum(), css.best_sum

In [ ]:
out_path = "data/" + name + f" (naive, {tol}, {cell_error}).xlsx"
df.to_excel(out_path)